In [4]:
import newspaper
from newspaper import Article
from newspaper import fulltext # for use with scrapy

from bs4 import BeautifulSoup

In [5]:
import requests
import json
import math

## Get list of URLs for a given source, in a given date range

In [6]:
base_url = 'https://newsapi.org/'
key_params = {
    'apiKey': '0db63d897c9844629bc477ab2639e451',
}

### Top headlines

In [13]:
url = ('https://newsapi.org/v2/top-headlines?'
#        'country=us&'
       'sources=bbc-news&'
#       'category=general&' # business entertainment general health science sports technology 
       'apiKey=0db63d897c9844629bc477ab2639e451')
response = requests.get(url)

print(json.dumps(response.json(), indent=2))

{
  "status": "ok",
  "totalResults": 10,
  "articles": [
    {
      "source": {
        "id": "bbc-news",
        "name": "BBC News"
      },
      "author": "BBC News",
      "title": "US faces backlash if car tariffs go ahead",
      "description": "The US is considering tariffs on foreign cars but the EU, Mexico and Canada say they would retaliate.",
      "url": "http://www.bbc.co.uk/news/business-44894417",
      "urlToImage": "https://ichef.bbci.co.uk/news/1024/branded_news/F029/production/_102618416_hi047877269.jpg",
      "publishedAt": "2018-07-19T21:02:22Z"
    },
    {
      "source": {
        "id": "bbc-news",
        "name": "BBC News"
      },
      "author": "BBC News",
      "title": "Trump in unusual criticism of Fed",
      "description": "Mr Trump says he is \"not happy\" with rising interest rates which he argues put the US at a disadvantage.",
      "url": "http://www.bbc.co.uk/news/business-44894415",
      "urlToImage": "https://ichef.bbci.co.uk/news/1024/bran

### Sources

In [8]:
import itertools as itr

# all_the_news sources
# existing_sources = {
#     'left':          ['The Atlantic','Buzzfeed','Vox',],
#     'center_left':   ['The Guardian','Washington Post','New York Times','CNN',],
#     'center':        ['NPR','Reuters'],
#     'center_right':  ['Wall Street Journal','The Hill'],
#     'right':         ['National Review','New York Post',],
#     'hyper_right':   ['Fox News','Breitbart'],
# }

sources = {
#     'hyper_left':    ['Occupy Democrats','Daily Kos'],
    'left':          ['MSNBC','Buzzfeed'], #['The Atlantic','Vox'],
    'center_left':   ['Politico','The Washington Post','The New York Times','CNN',],
    'center':        ['Reuters','Associated Press'], # ['NPR']
    'center_right':  ['The Wall Street Journal','The Hill'],
    'right':         ['National Review'], #['New York Post','The Weekly Standard','Examiner'],
    'hyper_right':   ['Fox News','Breitbart News','The American Conservative'],
}

sources_list = list(itr.chain(*sources.values()))
# NPR isn't that important - have plenty of centrist docs
# MUST look into 'The Atlantic','Vox','New York Post','The Weekly Standard','Examiner','Occupy Democrats','Daily Kos'
# or find replacements
# get centrist stuff started first

In [39]:
endpoint = '/v2/sources'
url = base_url + endpoint

params = {
    'country':'us',
    'language':'en',
#     'category': 'general',
}

response = requests.get(url, {**params, **key_params})

print(response.status_code)
# print(json.dumps(response.json(), indent=2))
print([(s['id'],s['name']) for s in response.json()['sources']])


# print([(s['id'],s['name']) for s in response.json()['sources']] if s['name'] in sources)

200
[('abc-news', 'ABC News'), ('al-jazeera-english', 'Al Jazeera English'), ('ars-technica', 'Ars Technica'), ('associated-press', 'Associated Press'), ('axios', 'Axios'), ('bleacher-report', 'Bleacher Report'), ('bloomberg', 'Bloomberg'), ('breitbart-news', 'Breitbart News'), ('business-insider', 'Business Insider'), ('buzzfeed', 'Buzzfeed'), ('cbs-news', 'CBS News'), ('cnbc', 'CNBC'), ('cnn', 'CNN'), ('crypto-coins-news', 'Crypto Coins News'), ('engadget', 'Engadget'), ('entertainment-weekly', 'Entertainment Weekly'), ('espn', 'ESPN'), ('espn-cric-info', 'ESPN Cric Info'), ('fortune', 'Fortune'), ('fox-news', 'Fox News'), ('fox-sports', 'Fox Sports'), ('google-news', 'Google News'), ('hacker-news', 'Hacker News'), ('ign', 'IGN'), ('mashable', 'Mashable'), ('medical-news-today', 'Medical News Today'), ('msnbc', 'MSNBC'), ('mtv-news', 'MTV News'), ('national-geographic', 'National Geographic'), ('national-review', 'National Review'), ('nbc-news', 'NBC News'), ('new-scientist', 'New Sc

In [10]:
print(len(sources_list))
my_sources = [s for s in response.json()['sources'] if s['name'] in sources_list]

for i,s in enumerate(my_sources):
    print(i,s)

14
0 {'id': 'associated-press', 'name': 'Associated Press', 'description': 'The AP delivers in-depth coverage on the international, politics, lifestyle, business, and entertainment news.', 'url': 'https://apnews.com/', 'category': 'general', 'language': 'en', 'country': 'us'}
1 {'id': 'breitbart-news', 'name': 'Breitbart News', 'description': 'Syndicated news and opinion website providing continuously updated headlines to top news and analysis sources.', 'url': 'http://www.breitbart.com', 'category': 'general', 'language': 'en', 'country': 'us'}
2 {'id': 'buzzfeed', 'name': 'Buzzfeed', 'description': 'BuzzFeed is a cross-platform, global network for news and entertainment that generates seven billion views each month.', 'url': 'https://www.buzzfeed.com', 'category': 'entertainment', 'language': 'en', 'country': 'us'}
3 {'id': 'cnn', 'name': 'CNN', 'description': 'View the latest news and breaking news today for U.S., world, weather, entertainment, politics and health at CNN', 'url': 'h

### Everything endpoint

In [1]:
import requests
import json
import time
import math
import itertools as itr
from ediblepickle import checkpoint

import pandas as pd

In [56]:
base_url = 'https://newsapi.org/'
key_params = {
    'apiKey': '0db63d897c9844629bc477ab2639e451',
}

endpoint = 'v2/everything'
url = base_url + endpoint

def get_article_data(source, date_range):
    date_params = {
        'from': date_range[0], 
        'to': date_range[1], 
    }

    page_size = 100
    page = 1
    max_page = (10000 / page_size)
    n_pages = None

    articles = []

    while n_pages is None or page < n_pages + 1:
        other_params = {
            'sources': source['id'],
            'language': 'en',
            'pageSize': page_size,
            'page': page,
        }

        print('Page: ', page)
        params = {**date_params, **other_params, **key_params}

        response = requests.get(url, params)
#         print(response.url)
        json_resp = response.json()
        
        if json_resp['status'] == 'ok':

            if n_pages is None:
                total_results = json_resp['totalResults']
                print(total_results)
                n_pages = math.ceil(total_results / page_size)
                print(n_pages)

                if max_page < n_pages:
                    print('Unable to retrieve all articles from {source} in range ({from}-{to}). Getting first 10k.'
                          .format(source=source, **date_params))

        page += 1

        articles.extend(json_resp['articles'])

    return articles


def preload_urls(source, date_range):
    time.sleep(0.01)  # TODO neeeedeD?
    articles = get_article_data(source, date_range)
    pd.DataFrame(articles).to_csv('articles.csv', index=False)

In [57]:
preload_urls(my_sources[2], ('2018-06-01','2018-07-18'))

Page:  1
4484
45
Page:  2
Page:  3
Page:  4
Page:  5
Page:  6
Page:  7
Page:  8
Page:  9
Page:  10
Page:  11
Page:  12
Page:  13
Page:  14
Page:  15
Page:  16
Page:  17
Page:  18
Page:  19
Page:  20
Page:  21
Page:  22
Page:  23
Page:  24
Page:  25
Page:  26
Page:  27
Page:  28
Page:  29
Page:  30
Page:  31
Page:  32
Page:  33
Page:  34
Page:  35
Page:  36
Page:  37
Page:  38
Page:  39
Page:  40
Page:  41
Page:  42
Page:  43
Page:  44
Page:  45


In [58]:
articles_df = pd.read_csv('articles.csv')

In [42]:
response = requests.get(url, {'from': '2017-06-01',
                              'to': '2017-07-18', 
                              'sources': 'fox-news', 
                              'language': 'en', 
                              'pageSize': 10, 
#                               'page': 1, 
                              'apiKey': '0db63d897c9844629bc477ab2639e451'})
# requests.get('https://newsapi.org/v2/everything?q=apple&from=2018-07-18&to=2018-07-18&sortBy=popularity&apiKey=0db63d897c9844629bc477ab2639e451')
json_resp = response.json()
print(json_resp)

{'status': 'ok', 'sources': [{'id': 'abc-news', 'name': 'ABC News', 'description': 'Your trusted source for breaking news, analysis, exclusive interviews, headlines, and videos at ABCNews.com.', 'url': 'http://abcnews.go.com', 'category': 'general', 'language': 'en', 'country': 'us'}, {'id': 'al-jazeera-english', 'name': 'Al Jazeera English', 'description': 'News, analysis from the Middle East and worldwide, multimedia and interactives, opinions, documentaries, podcasts, long reads and broadcast schedule.', 'url': 'http://www.aljazeera.com', 'category': 'general', 'language': 'en', 'country': 'us'}, {'id': 'ars-technica', 'name': 'Ars Technica', 'description': "The PC enthusiast's resource. Power users and the tools they love, without computing religion.", 'url': 'http://arstechnica.com', 'category': 'technology', 'language': 'en', 'country': 'us'}, {'id': 'associated-press', 'name': 'Associated Press', 'description': 'The AP delivers in-depth coverage on the international, politics, l

limit_output extension: Maximum message size of 10000 exceeded with 16911 characters

## Newspaper scraping library

### Find categories

In [92]:
cnn_paper = newspaper.build('http://www.cnn.com')

for article in cnn_paper.articles[:10]:
    print(article.url)

http://www.cnn.com/election/2018/key-races
http://www.cnn.com/2017/03/04/vr/how-to-watch-vr
https://www.cnn.com/travel/article/giant-avocado-australia/index.html
https://www.cnn.com/travel/article/amsterdam-bridge-hotels/index.html
https://www.cnn.com/travel/article/huangguoshu-waterfalls-guizhou-china/index.html
https://www.cnn.com/travel/article/worlds-best-ancient-cities/index.html
https://www.cnn.com/travel/article/evolve-back-coorg-india/index.html
http://www.cnn.com/travel/article/cheesemonger-invitational-brooklyn-new-york/index.html
http://www.cnn.com/travel/article/dubai-vertical-farm-emirates-catering/index.html
http://www.cnn.com/travel/article/london-restaurants-great-views/index.html


In [88]:
for category in cnn_paper.category_urls():
    print(category)

http://cnn.com
http://cnn.com/travel
http://tours.cnn.com
http://cnn.com/accessibility
http://money.cnn.com
http://cnn.com/election
http://cnn.com/politics
http://cnn.com/vr
http://cnn.com/more
http://cnn.com/health
http://cnn.com/opinions
http://cnn.com/entertainment
http://cnn.com/europe
http://cnn.com/americas
http://cnn.com/asia
http://cnn.com/africa
http://cnn.com/us
http://cnn.com/world
http://cnn.com/middle-east
http://cnn.it
http://cnn.com/videos
http://cnn.com/uk
http://cnn.com/style
http://cnn.com/australia
http://collection.cnn.com
http://cnn.com/transcripts


In [100]:
newspaper.build('http://cnn.com/us',language='en').articles

[]

### Get text from URL

In [112]:
# html = requests.get(url).text
# text = fulltext(html)
# print(text)

In [115]:
soup = BeautifulSoup(requests.get(url).text, 'lxml')

In [109]:
url = urls[5]
# url = cnn_paper.articles[10].url
article = Article(url)
article.download()
# article.html

In [110]:
article.parse()
print(article.text)

Ruth Ben-Ghiat is a frequent contributor to CNN Opinion, and professor of history and Italian studies at New York University. Follow her on Twitter: @ruthbenghiat. The opinions expressed in this commentary are solely those of the author; view more opinion articles on CNN.

Anyone who needs confirmation of the secret of a strongman -- that he's a weak man, in constant need of ego affirmation -- need only look at images of the Helsinki "summit" between American President Donald Trump and Russian President Vladimir Putin, which revealed the insecurities of both men underneath the façade of power, and Trump's submissiveness to Putin in particular.

Ruth Ben-Ghiat

Here is a selection of those images captured by photo journalists. Of course, they don't convey every moment of the meeting, but they are consistent with what we saw on air during their appearances together. There are many other such similar images -- each seemingly more awkward than the last -- in which the subjects tell more of

In [176]:
print(article.title)
print(article.authors)
print(article.publish_date)
print(article.top_image)
print(article.movies)

Trump and Putin: the pictures tell the story
['Ruth Ben-Ghiat']
2018-07-18 00:00:00
https://cdn.cnn.com/cnnnext/dam/assets/180717133121-05-trump-putin-summit-0716-opinion-super-tease.jpg
[]


In [72]:
article.nlp()

In [73]:
print(article.keywords)
print(article.summary)

['wage', 'national', 'obamacare', 'leave', 'drones', 'minimum', 'law', 'laws', 'pot', 'guns', 'family', 'states', 'state']
× New Year, new laws: Obamacare, pot, guns and dronesBy Leigh Ann CaldwellWASHINGTON (CNN) — Not everyone subscribes to a New Year’s resolution, but Americans will be required to follow new laws in 2014.
Oregon: Family leave in Oregon has been expanded to allow eligible employees two weeks of paid leave to handle the death of a family member.
Minimum wage and former felon employmentWorkers in 13 states and four cities will see increases to the minimum wage.
New Jersey residents voted to raise the state’s minimum wage by $1 to $8.25 per hour.
California is also raising its minimum wage to $9 per hour, but workers must wait until July to see the addition.


In [178]:
type(article.download_exception_msg)

NoneType

In [ ]:
article.